## Exercise: Getting started with `acados` in `Python`
Authors: Jonathan Frey, Andrea Ghezzi, Moritz Diehl

### Important Resources:
- https://docs.acados.org/
- Installation guide https://docs.acados.org/installation/
- `Python` Interface: Installation and documentation https://docs.acados.org/python_interface/
    - Installation instructions https://docs.acados.org/installation
- `acados` problem formulation PDF: https://github.com/acados/acados/blob/master/docs/problem_formulation/problem_formulation_ocp_mex.pdf
- `acados` forum https://discourse.acados.org


### Exercise
We consider the nonlinear dynamical system given by a pendulum on a cart illustrated in the Figure below.
The state is given by $x = [p, \theta, v, \omega]^\top$ with cart position $p$, cart velocity $s$, angle of the pendulum $\theta$ and angular velocity $\omega$.
The control input is the horizontal force $F$ acting on the cart.
The ODE describing the system behavior is given by
\begin{align}
    \dot{x} = \begin{pmatrix}
    v \\
    \omega \\
    \dfrac{-m l \sin{(\theta)} \omega^2 + m g \cos{(\theta)} \sin{(\theta)}+F}{D} \\
    \dfrac{-m l \cos{(\theta)} \sin{(\theta)} \omega^2 + F \cos{(\theta)}+(M+m) g \sin{(\theta)}}{l D}
    \end{pmatrix}
    \text{with} \quad D = M + m - m\cos{(\theta)}^2
\end{align}

<img src="imgs/cartpole_schematic.png" alt="350" width="350"/>


#### Task overview
We aim to stabilize the pendulum in the upward position (unstable equilibrium point) using nonlinear model predictive control.

1. Fill out the function that exports the dynamic model of the pendulum
2. Run the open-loop simulation and plot the figure. If the dynamic is correct you should be able to stabilize the pendulum. **This step is useful to make you confortable we `acados` syntax**
3. Implement a closed-loop control of the system.
4. Modify the pendulum model to make the mass $M$ of the pendulum a parameter. Vary the mass if the cart only for the integrator by 20% and run again the closed-loop simulation. **The idea is to show the effect of model-plant mismatch**

In [ ]:
import numpy as np
from acados_template import AcadosModel, AcadosOcp, AcadosOcpSolver, AcadosSimSolver
import numpy as np
import scipy.linalg as scipylinalg
from utils import plot_pendulum


**Task 1**: Define a function that export the dynamic model of the system as an AcadosModel class.

In [ ]:
from casadi import SX, vertcat, sin, cos, Function

def export_pendulum_ode_model() -> AcadosModel:

    model_name = 'pendulum_ode'
    # constants
    M = 1. # mass of the cart [kg]
    m = 0.1 # mass of the ball [kg]
    g = 9.81 # gravity constant [m/s^2]
    l = 0.8 # length of the rod [m]

    # set up states & controls
    # FILL YOUR CODE HERE
    p = ...
    theta = ...
    v = ...
    dtheta = ...

    x = ...  # STACK THE STATES IN A VECTOR

    F = SX.sym('F')
    u = ...  # RENAME F WITH USUAL CONTROL NOTATION

    # xdot
    # FILL YOUR CODE HERE
    p_dot = ...
    theta_dot = ...
    v_dot = ...
    dtheta_dot = ...

    xdot = ... # STACK THE DOT QUANTITIES IN A VECTOR

    # algebraic variables
    # z = None

    # parameters
    p = []

    # dynamics
    # DEFINE THE ODE, MIGHT BE USEFUL TO DECLARE INTERMEDIATE QUANTITIES
    f_expl = vertcat(... ,
                     ... ,
                     ... ,
                     ...
                     )

    f_impl = xdot - f_expl

    # import the acados model and assign to it the required fields
    model = AcadosModel()

    model.f_impl_expr = f_impl
    model.f_expl_expr = f_expl
    model.x = x
    model.xdot = xdot
    model.u = u
    # model.z = z
    model.p = p
    model.name = model_name

    return model

**Task 2**: Now, you have to instanciate an `AcadosOcp` object and fill out the required field. \
If you do everything right, the OCP solution should be a stabilizing trajectory.
Pay attention to the cost formulation with the nonlinear least square module. You can checkout the syntax in Section 3.4 of https://github.com/acados/acados/blob/master/docs/problem_formulation/problem_formulation_ocp_mex.pdf

In [ ]:
# Setting the initial condition and the parameter of the simulation

X0 = np.array([0.0, np.pi/6, 0.0, 0.0])
N_horizon = 50
T_horizon = 2.0
Fmax = 80

In [ ]:
# create ocp object to formulate the OCP
ocp = AcadosOcp()

# set model
model = export_pendulum_ode_model()
ocp.model = model

# set dimensions
ocp.dims.N = N_horizon
nx = ocp.model.x.size()[0]
nu = ocp.model.u.size()[0]

# set cost
Q_mat = ...
R_mat = ...

# We use the NONLINEAR_LS cost type and GAUSS_NEWTON Hessian approximation - One can also use the external cost module to specify generic cost.
ocp.cost.cost_type = 'NONLINEAR_LS'
ocp.cost.cost_type_e = 'NONLINEAR_LS'
ocp.cost.W = scipylinalg.block_diag(...)
ocp.cost.W_e = Q_mat

ocp.model.cost_y_expr = ...
ocp.model.cost_y_expr_e = ...
ocp.cost.yref = np.zeros(...)
ocp.cost.yref_e = ...

# set constraints
ocp.constraints.lbu = ...
ocp.constraints.ubu = ...
ocp.constraints.idxbu = ...

ocp.constraints.x0 = ...

# set options
ocp.solver_options.qp_solver = 'PARTIAL_CONDENSING_HPIPM' # FULL_CONDENSING_QPOASES, PARTIAL_CONDENSING_HPIPM
ocp.solver_options.hessian_approx = 'GAUSS_NEWTON' # 'GAUSS_NEWTON', 'EXACT'
ocp.solver_options.integrator_type = 'IRK'
# ocp.solver_options.print_level = 1
ocp.solver_options.nlp_solver_type = 'SQP' # SQP_RTI, SQP
ocp.solver_options.nlp_solver_max_iter = 400
# ocp.solver_options.levenberg_marquardt = 1e-4

# set prediction horizon
ocp.solver_options.tf = ...

In [ ]:
# Create OCP solver
acados_ocp_solver = AcadosOcpSolver(ocp)

# Initialize quantities
nx = ocp.model.x.size()[0]
nu = ocp.model.u.size()[0]
simX = np.ndarray((N_horizon+1, nx))
simU = np.ndarray((N_horizon, nu))

# Solve OCP by calling the method .solve()
status = acados_ocp_solver.solve()
acados_ocp_solver.print_statistics()  #NOTE: encapsulates: stat = acados_ocp_solver.get_stats("statistics")

if status != 0:
    raise Exception(f'acados returned status {status}.')

# get solution
for i in range(N_horizon):
    simX[i,:] = acados_ocp_solver.get(...)
    simU[i,:] = acados_ocp_solver.get(...)
simX[N_horizon,:] = acados_ocp_solver.get(N_horizon, "x")

plot_pendulum(np.linspace(0, T_horizon, N_horizon+1), Fmax, simU, simX, latexify=False)

----
**Task 3**: Let's implement the closed loop control of our cart pole system.
1. We need an acados ocp solver, to solve the ocp, and an acados integrator, to simulate the system in closed-loop
2. Remember: in closed-loop simulation we need to update the new initial state of the OCP -> let's use the `solve_for_x0` method
3. Remember: we apply to the system only the fist optimal control input $u_{0|t}^\star$ -> let's use the `simulate` method


In [ ]:
# create an integrator with the same settings as used in the OCP solver.
acados_integrator = AcadosSimSolver(ocp)

Nsim = 100
simX = np.ndarray((Nsim+1, nx))
simU = np.ndarray((Nsim, nu))

simX[0,:] = ...

# closed loop
for i in range(Nsim):

    # initialize solver
    if i == 0:
        for stage in range(N_horizon+1):
            acados_ocp_solver.set(stage, 'x', ...)

    # solve ocp
    ... acados_ocp_solver.solve_for_x0(...)

    # simulate system and update state in simX
    ... = acados_integrator.simulate(...)


# plot results
plot_pendulum(np.linspace(0, T_horizon/N_horizon*Nsim, Nsim+1), Fmax, simU, simX)

---
**Task 4: model-plant mismatch**
1. Copy-paste the cell with the model and edit it such that $M$ is a parameter of the OCP. In this way you can easily vary it.
2. Vary the mass by a 20% and run again the closed loop simulation. What do you observe?

In [ ]:
# This cell is needed to avoid conflicts among due to python notebooks
acados_ocp_solver = None
acados_integrator = None

In [ ]:
# COPY-PASTE HERE THE MODEL AND EDIT IT TO MAKE IT PARAMETRIC IN M

In [ ]:
# RECREATE THE OCP, REMEMBER TO SET `ocp.parameter_values` TO THE DEFAULT PARAMETER

In [ ]:
# COPY-PASTE HERE THE CLOSED-LOOP SIMULATION AND REMEMBER TO CHANGE THE VALUE OF M FOR THE ACADOS INTEGRATOR